In [1]:
with open('../Data/10_computable_moments.txt', 'r') as file:
    content = file.read()

In [2]:
import json

# Create a list to hold the JSON objects
json_list = []

# Iterate over the lines and create the JSON objects
lines = content.split('\n')
# Remove the first line
lines = lines[1:]

for idx, line in enumerate(lines):
    if line:
        parts = line.split()
        json_obj = {
            "id": [idx],
            "type": parts[0],
            "tagLength": parts[1],
            "tags": parts[2:]
        }
        json_list.append(json_obj)

# Convert the list to a JSON string
json_str = json.dumps(json_list, indent=4)

In [3]:
Portrait_Images = [i for i in json_list if i["type"] == "P"]
Landscape_Images = [i for i in json_list if i["type"] == "L"]

In [4]:
def min_form_number_of_common_tags(row1, row2):

    set1 = set(row1['tags'])
    set2 = set(row2['tags'])
    
    tags_in_row1_not_in_row2 = len(set1 - set2)
    tags_in_row2_not_in_row1 = len(set2 - set1)
    common_tags = len(set1 & set2)
    
    return min(tags_in_row1_not_in_row2, tags_in_row2_not_in_row1, common_tags)


In [5]:
Frame_Portrait_Images = []

if(len(Portrait_Images)>0):
    
    Temp_Portrait_Images = Portrait_Images.copy()
    
    for i in range(len(Portrait_Images)):
        if len(Temp_Portrait_Images) == 0:
            break
        
        element1 = Temp_Portrait_Images.pop(0)
        
        for j in range(len(Temp_Portrait_Images)):
            element2 = Temp_Portrait_Images[j]
            common_tags = len(set(element1.get('tags')) & set(element2.get('tags')))
            if common_tags < 1:
                combined_tags = list(set(element1.get('tags')) | set(element2.get('tags')))
                Frame_Portrait_Images.append({
                    "id": [element1['id'][0], element2['id'][0]],
                    "type": "P",
                    "tagLength": len(combined_tags),
                    "tags": combined_tags
                })
                Temp_Portrait_Images.pop(j)
                break
            else :
                if j == len(Temp_Portrait_Images) - 1:
                    combined_tags = list(set(element1.get('tags')) | set(element2.get('tags')))
                    Frame_Portrait_Images.append({
                        "id": [element1['id'][0], element2['id'][0]],
                        "type": "P",
                        "tagLength": len(combined_tags),
                        "tags": combined_tags
                    })
                    Temp_Portrait_Images.pop(j)
                    break

In [6]:
Final_List = Frame_Portrait_Images + Landscape_Images

In [7]:
Tag_List = []

for i in range(len(Final_List)):
    for tag in Final_List[i]['tags']:
        
        tag_index = next((index for (index, d) in enumerate(Tag_List) if d["tag"] == tag), None)
        if tag_index is None:
            Tag_List.append({"tag": tag, "id": [i]})
        else:
            Tag_List[tag_index]["id"].append(i)
            

In [8]:
print(Tag_List)

[{'tag': 'k81', 'id': [0, 16, 22, 51, 133, 134, 136, 143, 155, 190, 210, 214, 231, 250, 377, 391, 466, 577, 583, 658]}, {'tag': 'k17', 'id': [0, 603]}, {'tag': 'k201', 'id': [0]}, {'tag': 'k8b1', 'id': [0, 53, 87, 102, 108, 175, 192, 229, 243, 249, 251, 259, 282, 351, 356, 506, 530, 538, 618, 687]}, {'tag': 'kjb1', 'id': [0, 8, 27, 35, 56, 59, 83, 107, 115, 135, 141, 193, 206, 213, 217, 248, 318, 373, 598, 633]}, {'tag': 'kb6', 'id': [0]}, {'tag': 'kg11', 'id': [0, 318, 384]}, {'tag': 'kw52', 'id': [0, 10, 114, 282, 310, 379, 441, 452, 538, 544]}, {'tag': 'k652', 'id': [0]}, {'tag': 'k001', 'id': [0, 2, 18, 31, 59, 80, 117, 180, 214, 358, 369, 420, 433, 482, 497, 514, 578, 605, 620, 691]}, {'tag': 'kd7', 'id': [0, 3, 29, 46, 47, 83, 106, 107, 183, 200, 216, 239, 261, 276, 278, 302, 428, 655, 701, 738]}, {'tag': 'k1l', 'id': [0, 353]}, {'tag': 'k4q', 'id': [0, 249]}, {'tag': 'kg6', 'id': [0, 28, 653]}, {'tag': 'kmz1', 'id': [0, 31, 166]}, {'tag': 'k9c1', 'id': [1, 14, 60, 134, 169, 170,

In [9]:
Used_images = [0]

Sorted_Images = []

Temp_Sorted_List = Final_List.copy()

if Temp_Sorted_List:
    Sorted_Images.append(Temp_Sorted_List.pop(0))

for i in range(len(Final_List)-1):
    element1 = Sorted_Images[-1]
    
    if not Temp_Sorted_List:
        break
    
    all_tag = []
    
    for tags in element1['tags']:
        temp_hold_tags = [i for i in Tag_List if tags in i["tag"]]
        all_tag.append(temp_hold_tags[0])
            
    target_tag = [i for i in all_tag if len(i['id']) > 1]
    
    target_images = []
    
    for images in target_tag:
        for i in images['id']:
            Image_index = next((index for (index, d) in enumerate(target_images) if d["id"] == i ), None)
            
            if i in Used_images:
                continue
            else:
                if Image_index is None:
                    target_images.append({"id":i, "count":1})
                else:
                    target_images[Image_index]["count"] += 1
            
    for i in range(len(target_images)):
        target_images[i]['count'] = min_form_number_of_common_tags(element1, Final_List[target_images[i]['id']])
        
    target_images = sorted(target_images, key = lambda i: i['count'], reverse = True)
    
    if len(target_images) == 0:
        
        Used_images.append(Final_List.index(Temp_Sorted_List[0]))
    
        Sorted_Images.append(Temp_Sorted_List[0])
    
        Temp_Sorted_List.pop(0)
        
    else:
        
        Used_images.append(target_images[0]['id'])
        
        Sorted_Images.append(Final_List[target_images[0]['id']])
        
        Temp_Sorted_List.pop(Temp_Sorted_List.index(Final_List[target_images[0]['id']]))
    

In [10]:
print(element1)

print(Temp_Sorted_List)

print(Used_images)

{'id': [976], 'type': 'L', 'tagLength': '6', 'tags': ['k6n1', 'k2', 'khl1', 'kd51', 'k66', 'k2z1']}
[]
[0, 214, 18, 95, 76, 210, 231, 209, 24, 73, 16, 559, 605, 71, 185, 101, 123, 54, 69, 96, 207, 23, 99, 406, 57, 84, 236, 28, 44, 243, 10, 204, 33, 61, 102, 88, 203, 202, 29, 119, 60, 145, 208, 15, 744, 198, 157, 168, 1, 127, 256, 188, 103, 121, 196, 245, 156, 169, 226, 70, 164, 227, 128, 113, 211, 42, 189, 9, 171, 179, 77, 240, 239, 474, 63, 49, 104, 457, 418, 639, 218, 213, 59, 122, 212, 68, 7, 39, 516, 181, 114, 186, 248, 20, 634, 115, 217, 26, 34, 150, 91, 216, 183, 655, 289, 389, 197, 277, 46, 8, 159, 118, 205, 48, 53, 175, 148, 165, 294, 132, 426, 35, 141, 206, 194, 176, 238, 100, 50, 272, 537, 650, 667, 110, 385, 82, 166, 692, 665, 133, 190, 428, 5, 233, 36, 93, 31, 162, 720, 424, 2, 45, 87, 540, 74, 125, 111, 246, 404, 112, 305, 124, 147, 192, 108, 126, 587, 37, 446, 158, 78, 130, 135, 163, 142, 131, 152, 627, 460, 64, 139, 140, 6, 40, 309, 727, 619, 454, 41, 85, 244, 229, 544, 

In [11]:
testing = min_form_number_of_common_tags(Final_List[0], Final_List[31])
print(testing)

2


In [12]:
def calculate_score(frameglasses):
    """Calculates the Global Robotic Satisfaction."""
    score = 0
    for i in range(len(frameglasses) - 1):
        tags1 = frameglasses[i]
        tags2 = frameglasses[i + 1]
        
        local_score = min_form_number_of_common_tags(tags1, tags2)
        score += local_score

    return score

print(calculate_score(Sorted_Images))

1462


In [13]:
output_file_path = "../Data/output10.txt"

with open(output_file_path, 'w') as file:
    # Write the total number of rows
    file.write(f"{len(Sorted_Images)}\n")
    
    # Write the 'id' column of each row without brackets
    for row in Sorted_Images:
        ids_str = ' '.join(map(str, row.get('id')))
        file.write(f"{ids_str}\n")

print(f"Output file created at: {output_file_path}")

Output file created at: ../Data/output10.txt
